In [1]:
%load_ext autoreload
%autoreload 2

main_dir = '/Users/simon/bsose_monthly/'
salt = main_dir + 'bsose_i106_2008to2012_monthly_Salt.nc'
theta = main_dir + 'bsose_i106_2008to2012_monthly_Theta.nc'

In [2]:
import numpy as np
import xarray as xr
xr.set_options(keep_attrs=True)
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature
import matplotlib.path as mpath
import pyxpcm
from pyxpcm.models import pcm


def pcm_pca_out(time_i=42, K=4, maxvar=2):  
    # Define features to use
    # Instantiate the PCM
    
    max_depth = 2000
    z = np.arange(0., -max_depth, -10.)
    features_pcm = {'THETA': z, 'SALT': z}
    features = {'THETA': 'THETA', 'SALT': 'SALT'}
    salt_nc = xr.open_dataset(salt).isel(time=time_i)
    theta_nc = xr.open_dataset(theta).isel(time=time_i)
    big_nc = xr.merge([salt_nc, theta_nc])
    both_nc = big_nc.where(big_nc.coords['Depth'] > 
                           max_depth).drop(['iter', 'Depth', 
                                            'rA', 'drF', 'hFacC'])   
 
    attr_d = {}

    for coord in both_nc.coords:
        attr_d[coord] = both_nc.coords[coord].attrs
        
    lons_new = np.linspace(both_nc.XC.min(), both_nc.XC.max(), 60*4)
    lats_new = np.linspace(both_nc.YC.min(), both_nc.YC.max(), 60)
    # ds = both_nc # .copy(deep=True)
    ds = both_nc.interp(coords={'YC': lats_new, 'XC': lons_new})#, method='cubic')
        
    m = pcm(K=K, features=features_pcm, 
            maxvar=maxvar, 
            timeit=True, timeit_verb=1)
    X = m.add_pca_to_xarray(ds, features=features, dim='Z')
    
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]
    
    return ds, m, X
    
def cut_off():
    m.fit(ds, features=features, dim='Z') #, inplace=True)
    m.predict(ds, features=features, dim='Z', inplace=True)
    m.predict_proba(ds, features=features, dim='Z', inplace=True)
    m.find_i_metric(ds, inplace=True)
    
    def sanitize():
        del ds.PCM_LABELS.attrs['_pyXpcm_cleanable']
        del ds.PCM_POST.attrs['_pyXpcm_cleanable']
        del ds.PCM_RANK.attrs['_pyXpcm_cleanable']
        
    for coord in attr_d:
        ds.coords[coord].attrs = attr_d[coord]
    
    sanitize()
    return ds, m

/Users/simon/pyxpcm/pyxpcm/plot.py:45: UserWarning: pyXpcm requires seaborn installed for full plotting functionality
  warnings.warn("pyXpcm requires seaborn installed for full plotting functionality")


In [3]:
ds, m, X = pcm_pca_out(K=5)
ds

  fit.1-preprocess.1-mask: 17 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02
 -5.515e+02 -6.140e+02 -7.000e+02 -8.000e+02 -9.000e+02 -1.000e+03
 -1.100e+03 -1.225e+03 -1.400e+03 -1.600e+03 -1.800e+03 -2.010e+03
 -2.270e+03 -2.610e+03 -3.000e+03 -3.400e+03 -3.800e+03 -4.200e+03
 -4.600e+03 -5.000e+03 -5.400e+03 -5.800e+03]
  fit.1-preprocess.2-feature_THETA.1-ravel: 20 ms
[-2.100e+00 -6.700e+00 -1.215e+01 -1.855e+01 -2.625e+01 -3.525e+01
 -4.500e+01 -5.500e+01 -6.500e+01 -7.500e+01 -8.500e+01 -9.500e+01
 -1.050e+02 -1.150e+02 -1.250e+02 -1.350e+02 -1.465e+02 -1.615e+02
 -1.800e+02 -2.000e+02 -2.200e+02 -2.400e+02 -2.600e+02 -2.800e+02
 -3.010e+02 -3.270e+02 -3.610e+02 -4.025e+02 -4.500e+02 -5.000e+02


<xarray.Dataset>
Dimensions:  (XC: 240, YC: 60, Z: 52)
Coordinates:
    time     datetime64[ns] 2011-08-01T15:12:00
  * Z        (Z) float32 -2.1 -6.7 -12.15 -18.55 ... -5000.0 -5400.0 -5800.0
  * YC       (YC) float64 -77.98 -77.16 -76.35 -75.53 ... -31.35 -30.53 -29.72
  * XC       (XC) float64 0.08333 1.589 3.094 4.6 ... 355.4 356.9 358.4 359.9
Data variables:
    SALT     (Z, YC, XC) float64 nan nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0
    THETA    (Z, YC, XC) float64 nan nan nan nan nan nan ... 0.0 0.0 0.0 0.0 0.0

In [31]:
type(X)

xarray.core.dataarray.DataArray

In [32]:
X

<xarray.DataArray (n_samples: 10437, n_features: 4)>
array([[-1.49857705e+00, -1.35610183e-03, -1.39455909e+00,
         1.92831994e-01],
       [-1.54176135e+00,  1.71316841e-03, -1.38697422e+00,
         1.62822443e-01],
       [-1.55224458e+00,  3.34145152e-04, -1.41139736e+00,
         2.70454772e-01],
       ...,
       [ 1.54223376e+00,  4.03170127e-01, -2.58781494e-01,
         1.20336443e+00],
       [ 1.60955931e+00,  4.16945466e-01, -2.62509270e-01,
         1.30807667e+00],
       [ 1.49253627e+00,  3.65197949e-01, -3.27807213e-01,
         1.09906200e+00]])
Coordinates:
  * n_samples   (n_samples) int64 0 1 2 3 4 5 ... 10432 10433 10434 10435 10436
  * n_features  (n_features) <U7 'THETA_0' 'THETA_1' 'SALT_0' 'SALT_1'

            with self._context('predict_proba.predict', self._context_args):
                post_values = self._classifier.predict_proba(X)

            with self._context('predict_proba.score', self._context_args):
                self._props['llh'] = self._classifier.score(X)

            # Create a xarray with posteriors:
            with self._context('predict_proba.xarray', self._context_args):
                P = list()
                for k in range(self.K):
                    X = post_values[:, k]
                    x = self.unravel(ds, sampling_dims, X)
                    P.append(x)
                
                da = xr.concat(P, dim=classdimname).rename(name)
                da.attrs['long_name'] = 'PCM posteriors'
                da.attrs['units'] = ''
                da.attrs['valid_min'] = 0
                da.attrs['valid_max'] = 1
                da.attrs['llh'] = self._props['llh']

            # Add posteriors to the dataset:
            if inplace:
                return ds.pyxpcm.add(da)
            else:
                return da
